# 3. Baseline Models - Gold Price Forecasting

**Objective:** Implement and evaluate baseline machine learning models.

**Author:** Félix Jouary  
**Dataset:** Kaggle Gold Price Dataset

**Models in this notebook:**
- Linear Regression (baseline)
- Ridge Regression
- Lasso Regression
- Decision Tree
- Random Forest
- Gradient Boosting

## 3.1 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# Sklearn models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")

## 3.2 Load Processed Data

In [ ]:
# Load scaled data
X_train_scaled = np.load('../data/processed/X_train_scaled.npy')
X_test_scaled = np.load('../data/processed/X_test_scaled.npy')
y_train = np.load('../data/processed/y_train.npy')
y_test = np.load('../data/processed/y_test.npy')

# Load feature names
feature_names = pd.read_csv('../data/processed/feature_names.csv').iloc[:, 0].tolist()

# Load original data for visualization
train_data = pd.read_csv('../data/processed/train_data.csv', parse_dates=['Date'])
test_data = pd.read_csv('../data/processed/test_data.csv', parse_dates=['Date'])

print(f"X_train shape: {X_train_scaled.shape}")
print(f"X_test shape: {X_test_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"\nNumber of features: {len(feature_names)}")

## 3.3 Define Evaluation Metrics

For regression problems, we use:
- **RMSE** (Root Mean Squared Error): Penalizes large errors
- **MAE** (Mean Absolute Error): Average error magnitude
- **MAPE** (Mean Absolute Percentage Error): Percentage error
- **R²** (Coefficient of Determination): Explained variance

In [ ]:
def evaluate_model(y_true, y_pred, model_name="Model"):
    """Calculate and display regression metrics."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{'='*50}")
    print(f"{model_name} - Evaluation Metrics")
    print(f"{'='*50}")
    print(f"RMSE:  ${rmse:.2f}")
    print(f"MAE:   ${mae:.2f}")
    print(f"MAPE:  {mape:.2f}%")
    print(f"R²:    {r2:.4f}")
    
    return {'Model': model_name, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

# Store results for comparison
results = []

## 3.4 Model 1: Linear Regression (Baseline)

In [ ]:
# Train Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred_lr_train = lr_model.predict(X_train_scaled)
y_pred_lr_test = lr_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_lr_train, "Linear Regression (Train)")

print("\nTEST SET:")
lr_results = evaluate_model(y_test, y_pred_lr_test, "Linear Regression")
results.append(lr_results)

## 3.5 Model 2: Ridge Regression (L2 Regularization)

In [ ]:
# Hyperparameter tuning for Ridge
ridge_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Time Series Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

ridge_grid = GridSearchCV(
    Ridge(),
    ridge_params,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

ridge_grid.fit(X_train_scaled, y_train)

print(f"Best alpha: {ridge_grid.best_params_['alpha']}")
print(f"Best CV score (neg MSE): {ridge_grid.best_score_:.2f}")

In [ ]:
# Train with best parameters
ridge_model = ridge_grid.best_estimator_

# Predictions
y_pred_ridge_train = ridge_model.predict(X_train_scaled)
y_pred_ridge_test = ridge_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_ridge_train, "Ridge Regression (Train)")

print("\nTEST SET:")
ridge_results = evaluate_model(y_test, y_pred_ridge_test, "Ridge Regression")
results.append(ridge_results)

## 3.6 Model 3: Lasso Regression (L1 Regularization)

In [ ]:
# Hyperparameter tuning for Lasso
lasso_params = {'alpha': [0.01, 0.1, 1, 10, 100]}

lasso_grid = GridSearchCV(
    Lasso(max_iter=10000),
    lasso_params,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

lasso_grid.fit(X_train_scaled, y_train)

print(f"Best alpha: {lasso_grid.best_params_['alpha']}")
print(f"Best CV score (neg MSE): {lasso_grid.best_score_:.2f}")

In [ ]:
# Train with best parameters
lasso_model = lasso_grid.best_estimator_

# Predictions
y_pred_lasso_train = lasso_model.predict(X_train_scaled)
y_pred_lasso_test = lasso_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_lasso_train, "Lasso Regression (Train)")

print("\nTEST SET:")
lasso_results = evaluate_model(y_test, y_pred_lasso_test, "Lasso Regression")
results.append(lasso_results)

In [ ]:
# Feature importance from Lasso (non-zero coefficients)
lasso_coef = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lasso_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("Top 10 features by Lasso coefficient magnitude:")
print(lasso_coef.head(10))

# Count non-zero features
non_zero = (lasso_coef['Coefficient'] != 0).sum()
print(f"\nFeatures selected by Lasso: {non_zero}/{len(feature_names)}")

## 3.7 Model 4: Decision Tree

In [ ]:
# Hyperparameter tuning for Decision Tree
dt_params = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_grid = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    dt_params,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

dt_grid.fit(X_train_scaled, y_train)

print(f"Best parameters: {dt_grid.best_params_}")
print(f"Best CV score (neg MSE): {dt_grid.best_score_:.2f}")

In [ ]:
# Train with best parameters
dt_model = dt_grid.best_estimator_

# Predictions
y_pred_dt_train = dt_model.predict(X_train_scaled)
y_pred_dt_test = dt_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_dt_train, "Decision Tree (Train)")

print("\nTEST SET:")
dt_results = evaluate_model(y_test, y_pred_dt_test, "Decision Tree")
results.append(dt_results)

## 3.8 Model 5: Random Forest (Ensemble)

In [ ]:
# Hyperparameter tuning for Random Forest
rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    rf_params,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {rf_grid.best_params_}")
print(f"Best CV score (neg MSE): {rf_grid.best_score_:.2f}")

In [ ]:
# Train with best parameters
rf_model = rf_grid.best_estimator_

# Predictions
y_pred_rf_train = rf_model.predict(X_train_scaled)
y_pred_rf_test = rf_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_rf_train, "Random Forest (Train)")

print("\nTEST SET:")
rf_results = evaluate_model(y_test, y_pred_rf_test, "Random Forest")
results.append(rf_results)

In [ ]:
# Feature importance from Random Forest
rf_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Plot top 15 features
plt.figure(figsize=(10, 8))
plt.barh(rf_importance['Feature'].head(15)[::-1], rf_importance['Importance'].head(15)[::-1])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances - Random Forest')
plt.tight_layout()
plt.savefig('../reports/figures/rf_feature_importance.png', dpi=150)
plt.show()

print("Top 10 most important features:")
print(rf_importance.head(10))

## 3.9 Model 6: Gradient Boosting

In [ ]:
# Hyperparameter tuning for Gradient Boosting
gb_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_samples_split': [2, 5]
}

gb_grid = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    gb_params,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

gb_grid.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {gb_grid.best_params_}")
print(f"Best CV score (neg MSE): {gb_grid.best_score_:.2f}")

In [ ]:
# Train with best parameters
gb_model = gb_grid.best_estimator_

# Predictions
y_pred_gb_train = gb_model.predict(X_train_scaled)
y_pred_gb_test = gb_model.predict(X_test_scaled)

# Evaluate
print("TRAINING SET:")
_ = evaluate_model(y_train, y_pred_gb_train, "Gradient Boosting (Train)")

print("\nTEST SET:")
gb_results = evaluate_model(y_test, y_pred_gb_test, "Gradient Boosting")
results.append(gb_results)

## 3.10 Overfitting Analysis

In [ ]:
# Compare train vs test performance to detect overfitting
models_names = ['Linear Regression', 'Ridge', 'Lasso', 'Decision Tree', 'Random Forest', 'Gradient Boosting']
train_preds = [y_pred_lr_train, y_pred_ridge_train, y_pred_lasso_train, y_pred_dt_train, y_pred_rf_train, y_pred_gb_train]
test_preds = [y_pred_lr_test, y_pred_ridge_test, y_pred_lasso_test, y_pred_dt_test, y_pred_rf_test, y_pred_gb_test]

overfitting_analysis = []
for name, train_pred, test_pred in zip(models_names, train_preds, test_preds):
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    gap = ((test_rmse - train_rmse) / train_rmse) * 100
    
    overfitting_analysis.append({
        'Model': name,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'Gap (%)': gap
    })

overfit_df = pd.DataFrame(overfitting_analysis)
print("Overfitting Analysis (Train vs Test RMSE):")
print(overfit_df.to_string(index=False))
print("\nNote: Large positive gap indicates overfitting.")

In [ ]:
# Visualize overfitting
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(models_names))
width = 0.35

bars1 = ax.bar(x - width/2, overfit_df['Train RMSE'], width, label='Train RMSE')
bars2 = ax.bar(x + width/2, overfit_df['Test RMSE'], width, label='Test RMSE')

ax.set_xlabel('Model')
ax.set_ylabel('RMSE ($)')
ax.set_title('Train vs Test RMSE - Overfitting Analysis')
ax.set_xticks(x)
ax.set_xticklabels(models_names, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.savefig('../reports/figures/overfitting_analysis.png', dpi=150)
plt.show()

## 3.11 Model Comparison

In [ ]:
# Create comparison dataframe
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('RMSE')

print("Model Comparison (sorted by RMSE):")
print(results_df.to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RMSE
axes[0, 0].barh(results_df['Model'], results_df['RMSE'], color='steelblue')
axes[0, 0].set_xlabel('RMSE ($)')
axes[0, 0].set_title('RMSE by Model (lower is better)')

# MAE
axes[0, 1].barh(results_df['Model'], results_df['MAE'], color='coral')
axes[0, 1].set_xlabel('MAE ($)')
axes[0, 1].set_title('MAE by Model (lower is better)')

# MAPE
axes[1, 0].barh(results_df['Model'], results_df['MAPE'], color='seagreen')
axes[1, 0].set_xlabel('MAPE (%)')
axes[1, 0].set_title('MAPE by Model (lower is better)')

# R²
axes[1, 1].barh(results_df['Model'], results_df['R2'], color='purple')
axes[1, 1].set_xlabel('R²')
axes[1, 1].set_title('R² by Model (higher is better)')

plt.tight_layout()
plt.savefig('../reports/figures/model_comparison_baseline.png', dpi=150)
plt.show()

## 3.12 Predictions Visualization

In [ ]:
# Plot actual vs predicted for best model
best_model_name = results_df.iloc[0]['Model']
print(f"Best model: {best_model_name}")

# Get predictions from best model
if best_model_name == 'Random Forest':
    best_pred = y_pred_rf_test
elif best_model_name == 'Gradient Boosting':
    best_pred = y_pred_gb_test
elif best_model_name == 'Ridge Regression':
    best_pred = y_pred_ridge_test
elif best_model_name == 'Linear Regression':
    best_pred = y_pred_lr_test
elif best_model_name == 'Lasso Regression':
    best_pred = y_pred_lasso_test
else:
    best_pred = y_pred_dt_test

In [ ]:
# Time series plot
plt.figure(figsize=(14, 6))
plt.plot(test_data['Date'], y_test, label='Actual', alpha=0.8)
plt.plot(test_data['Date'], best_pred, label=f'Predicted ({best_model_name})', alpha=0.8)
plt.title(f'Gold Price Prediction - {best_model_name}')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.tight_layout()
plt.savefig('../reports/figures/best_model_predictions.png', dpi=150)
plt.show()

In [ ]:
# Scatter plot: Actual vs Predicted
plt.figure(figsize=(8, 8))
plt.scatter(y_test, best_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Price (USD)')
plt.ylabel('Predicted Price (USD)')
plt.title(f'Actual vs Predicted - {best_model_name}')
plt.legend()
plt.tight_layout()
plt.savefig('../reports/figures/actual_vs_predicted.png', dpi=150)
plt.show()

In [ ]:
# Residuals analysis
residuals = y_test - best_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals over time
axes[0].plot(test_data['Date'], residuals)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_title('Residuals Over Time')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Residual (USD)')

# Residuals distribution
axes[1].hist(residuals, bins=50, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_title('Residuals Distribution')
axes[1].set_xlabel('Residual (USD)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('../reports/figures/residuals_analysis.png', dpi=150)
plt.show()

print(f"Residuals Mean: {residuals.mean():.2f}")
print(f"Residuals Std: {residuals.std():.2f}")

## 3.13 Save Models and Results

In [ ]:
# Save all models
import os
os.makedirs('../models', exist_ok=True)

joblib.dump(lr_model, '../models/linear_regression.pkl')
joblib.dump(ridge_model, '../models/ridge_regression.pkl')
joblib.dump(lasso_model, '../models/lasso_regression.pkl')
joblib.dump(dt_model, '../models/decision_tree.pkl')
joblib.dump(rf_model, '../models/random_forest.pkl')
joblib.dump(gb_model, '../models/gradient_boosting.pkl')

# Save results
results_df.to_csv('../reports/baseline_results.csv', index=False)

print("All models saved successfully!")
print("\nFiles created:")
print("- ../models/linear_regression.pkl")
print("- ../models/ridge_regression.pkl")
print("- ../models/lasso_regression.pkl")
print("- ../models/decision_tree.pkl")
print("- ../models/random_forest.pkl")
print("- ../models/gradient_boosting.pkl")
print("- ../reports/baseline_results.csv")

## 3.14 Summary

### Baseline Models Implemented:

1. **Linear Regression** - Simple baseline
2. **Ridge Regression** - L2 regularization to prevent overfitting
3. **Lasso Regression** - L1 regularization for feature selection
4. **Decision Tree** - Non-linear model
5. **Random Forest** - Ensemble of decision trees
6. **Gradient Boosting** - Sequential ensemble learning

### Key Findings:

- All models use **TimeSeriesSplit** for cross-validation (respects temporal order)
- Hyperparameters were tuned using **GridSearchCV**
- Overfitting analysis performed comparing train vs test performance
- Feature importance analyzed for tree-based models

### Next Steps:

- Implement advanced models (XGBoost, LSTM)
- Try dimensionality reduction
- Final model comparison and selection

In [ ]:
# Final summary
print("="*60)
print("BASELINE MODELS SUMMARY")
print("="*60)
print(f"\nBest Model: {results_df.iloc[0]['Model']}")
print(f"Best RMSE: ${results_df.iloc[0]['RMSE']:.2f}")
print(f"Best MAE: ${results_df.iloc[0]['MAE']:.2f}")
print(f"Best MAPE: {results_df.iloc[0]['MAPE']:.2f}%")
print(f"Best R²: {results_df.iloc[0]['R2']:.4f}")
print("="*60)